#### Dinossauro

In [1]:
import pygame
cor_dino = 255,255,255
altura_dino = 40
largura_dino = 20
class Dinosaur:
    def __init__ (self, surfaceHeight):
        self.x = 60
        self.y = 0
        self.yvelocity = 0
        self.height = altura_dino
        self.width = largura_dino
        self.surfaceHeight = surfaceHeight
    def jump(self):
        if (self.y == 0): #Só pula se estiver no chao
            self.yvelocity = 300
    def update(self, deltaTime): # Update no pulo, vai mudando a posiçao de acordo com o tempo
        self.yvelocity += -500*deltaTime 
        self.y += self.yvelocity * deltaTime
        if self.y < 0: #volto pro chao e grudo no chao, volta pro normal
            self.y = 0
            self.yvelocity = 0

    def draw(self,display):
        pygame.draw.rect(display,cor_dino,[self.x,self.surfaceHeight-self.y-self.height,self.width,self.height])

    def check_collision(self,obstaculo):
        dino_rect = pygame.Rect(self.x, self.surfaceHeight- self.y -self.height, self.width,self.height)
        obstaculo_rect = pygame.Rect(obstaculo.x,obstaculo.y,obstaculo.width,obstaculo.height)

        return dino_rect.colliderect(obstaculo_rect)
        
            

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### Obstaculos

In [2]:
import pygame
class Obstaculos:
    def __init__(self,width,height,surface_width,surface_heigth, speed = 200):
        self.width = width
        self.height = height
        self.x = surface_width #Eixo X dos objetos
        self.y = surface_heigth - height  # Altura do chao
        self.speed = speed
    def update(self,deltaTime):
        self.x -= self.speed * deltaTime
    def draw(self,display,color=(255,0,0)): 
        pygame.draw.rect(display,color,[self.x,self.y+10,self.width,self.height-10])
    def should_delete(self):
        return self.x+self.width <0 #remove quando o eixo x e a soma da altura for menor que zero ou seja saiu da tela.

class Obstaculo_passaro:
    def __init__(self,width,height,surface_width,surface_heigth, speed = 200):
        self.width = width
        self.height = height
        self.x = surface_width #Eixo X dos objetos
        self.y = surface_heigth - height - 25  # Altura do chao
        self.speed = speed
    def update(self,deltaTime):
        self.x -= self.speed * deltaTime
    def draw(self,display,color=(255,0,0)): 
        pygame.draw.rect(display,color,[self.x,self.y-5,self.width,self.height-10])
    def should_delete(self):
        return self.x+self.width <0 #remove quando o eixo x e a soma da altura for menor que zero ou seja saiu da tela.
        
        


#### Pontos

In [3]:

class Pontos:
    def __init__(self,canto_x = 40, canto_y = 40):
        self.width = canto_x
        self.height = canto_y
        self.pontos = 0
        self.tempo = pygame.time.get_ticks()
        self.intervalo = 1000
    def update(self):
        tempo_atual = pygame.time.get_ticks()
        if (tempo_atual - self.tempo) >= self.intervalo:
            self.pontos += 1
            self.tempo = tempo_atual
        
    def draw(self,display):
        font = pygame.font.Font(None,36)
        text = font.render(f"pontos:{self.pontos}", True,( 255,255,255))
        display.blit(text,(self.width,self.height))

#### Base

In [4]:
import pygame
import random

pygame.init()

size = width, height = 640, 480
game_display = pygame.display.set_mode(size)

altura_chao = height - 100
preto = (0, 0, 0)
branco = (255, 255, 255)

dinosaur = Dinosaur(altura_chao)
lastFrame = pygame.time.get_ticks()

obstaculos = []
obstaculos_passaro = []
spawn_timer = 2.5  # Timer para spawn de cactos
spawn_timer_passaro = 3.5  # Timer para spawn de pássaros

running = True
game_over = False

pontos = Pontos()

while running:
    t = pygame.time.get_ticks()
    deltaTime = (t - lastFrame) / 1000
    lastFrame = t

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                dinosaur.jump()

    game_display.fill(preto)

    pygame.draw.rect(game_display, branco, [0, altura_chao, width, height - altura_chao])
    dinosaur.update(deltaTime)
    dinosaur.draw(game_display)

    # Spawn de cactos
    spawn_timer -= deltaTime
    if spawn_timer <= 0:
        obstaculos.append(Obstaculos(20, 40, width, altura_chao))
        spawn_timer = (2.5)  

    # Spawn de pássaros
    spawn_timer_passaro -= deltaTime
    if spawn_timer_passaro <= 0:
        # Verifica se há um cacto próximo
        overlap = False
        for obstaculo in obstaculos:
            if abs(obstaculo.x - width) < 50:  # Se um cacto está a menos de 50 pixels da posição X
                overlap = True
                break

        if not overlap:
            obstaculos_passaro.append(Obstaculo_passaro(20, 40, width, altura_chao))
        spawn_timer_passaro = 3.5  

    # Atualiza e desenha cactos
    obstaculos = [i for i in obstaculos if not i.should_delete()]
    for i in obstaculos:
        i.update(deltaTime)
        i.draw(game_display, color=(255, 0, 0))  # Cactos em vermelho

    # Atualiza e desenha pássaros
    obstaculos_passaro = [j for j in obstaculos_passaro if not j.should_delete()]
    for j in obstaculos_passaro:
        j.update(deltaTime)
        j.draw(game_display, color=(0, 0, 255))  # Pássaros em azul

    # Verifica colisão com cactos
    for obstaculo in obstaculos:
        if dinosaur.check_collision(obstaculo):
            game_over = True

    # Verifica colisão com pássaros
    for obstaculo_passaro in obstaculos_passaro:
        if dinosaur.check_collision(obstaculo_passaro):
            game_over = True

    # Game Over
    if game_over:
        print('Game Over! Pontuação:', pontos.pontos)
        obstaculos = []  # Limpa a lista de cactos
        obstaculos_passaro = []  # Limpa a lista de pássaros
        pontos.pontos = 0  # Zera a pontuação
        dinosaur = Dinosaur(altura_chao)  # Reinicia o dinossauro
        game_over = False  # Reseta o estado do jogo

    pontos.update()
    pontos.draw(game_display)

    pygame.display.update()

pygame.quit()

Game Over! Pontuação: 6
Game Over! Pontuação: 3
Game Over! Pontuação: 4
Game Over! Pontuação: 14
Game Over! Pontuação: 3
